# STRUCTURE-BASED VIRTUAL SCREENING (a guide)
#### Catalina Arnaiz - Bioinformatic Research Asisstant (CBGP, UPM-INIA)


_README_

_This guide is prepared to be run in either macOS or Linux, not in Windows. Executable files of the programs implemented are only available for these OS and thus, running this code in Windows will give rise to an error._

## 1. Protein Sanitization

In order to "sanitize" or clean the protein from undesireable ligands the LePhar molecular docking tool **LePro** will be implemented. LePro is designed to automatically add hydrogen atoms to proteins and/or nucleic acids by explicitely considering the protonation state of histidine. All crystal water, ions, small ligands and cofactors except HEM are also removed. 

LePro software download: http://www.lephar.com/software.htm 

In [7]:
! ./lepro_mac Output_docking\Protein1\Protein1.pdb
! ./lepro_linux_x86

************************************************************
*      LePro                                               *
*            Add hydrogen atoms to a protein &             *
*            write the input file for LeDock               *
*      Copyright (C) 2013-21 Hongtao Zhao, PhD             *
*      Email: htzhaovv@gmail.com                           *
************************************************************
----------Usage:                                                                       
          lepro [PDB file] [-rot || -metal || -p]                                        
          -rot  [[chain] resid] align principal axes of the binding site with Cartesian
          -metal keep ZN/MN/CA/MG                                                      
          -metal -p redistribute metal charge to protein                               



Now we will have a "clean" protein pdb file automatically saved as *pro.pdb* and a configuration file that can be used with LeDock (just another method). 

**NOTE**: the running of this cell in Windows is difficult because no actual LePro executable is available for this OS. Nonetheless, you can always prep your protein in Chimera by adding H (Tools > Structure Editing > Add H) and running a structure optimizaiton protocol: 500 steepest descent steps + 200 conjugate gradient steps (Tools > Structure Editing > Minimize Structure). Then proceed to eliminate any excess molecules present in the structure (Select > "whatever" + Actions > Atoms > delete). Instead you can run this from you Virtual Machine.

## 2. Ligand Sanitization - Ligand battery prep

The protein now has a clean structure and H added to it, so it is time to think about the ligands we want to dock onto our protein. Once a list of putative ligands/interactors has been created they need to be prepared for the docking protocol. This is a simple step that can be done using two Python packages *rdkit* and *openbabel*. 

**NOTE**: I encourage you to use VSCode to edit Notebooks, Python or even Ruby scripts. VSCode makes the instalation of packages very simple (at least for Python). I have experienced that working through Anaconda is easier, as it offers the possibility to create environments. I like to create environments for different "jobs", I like to think of it as a way to organize my work in a better way - for protein docking protocols I have created a new *conda environment* containing the two packages mentioned above. 

The following code should be run on the terminal of a VSCode window that has been opened through Anaconda Navigator:
```
!conda create -c rdkit -n my-rdkit-env rdkit
!conda env list #verify the availability of the environments
!conda activate my-rdkit-env
!conda install -c conda-forge openbabel -n my-rdkit-env  ##instal openbabel in your newly created rdkit environment
```

In [1]:
### Install packages
from openbabel import pybel
import pandas as pd
import rdkit.Chem.AllChem as AllChem 
import rdkit.Chem as Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
from rdkit import DataStructs
from rdkit.Chem import rdFingerprintGenerator
import math
import matplotlib.pyplot as plt

In order to create a file that can be read by the docking program we are going to:
1. Load each molecule from its SMILE onto mols
2. Add H to mols, creating hmols: remember H atoms are important for getting realistic geometries (embed molecules)
3. Store different bits of relevant information of each molecule such as the metabolite name, its numeric id on the provided library and the inchikey (very usefull for human data, not so much for plant data)
4. Create an sdf file (more info on sdf files here: https://www.nonlinear.com/progenesis/sdf-studio/v0.9/faq/sdf-file-format-guidance.aspx)

In [7]:
#load excel file to variable containing for example, metabolomic results 
df = pd.read_csv('./Databases/qsar_ro5_final_database.csv', header=0) 
print(f"Number of molecules: {df.shape[0]}\n"+
      f"Column names: {df.columns}")
df.head(3)

Number of molecules: 16
Column names: Index(['CanonicalSMILES', 'CID', 'ROMol', 'tanimoto_maccs', 'dice_maccs',
       'tanimoto_morgan', 'dice_morgan', 'MACCS_fp', 'pIC50_pred', 'IC50_pred',
       'molecular_weight', 'n_hba', 'n_hbd', 'logp', 'TPSA', 'ro5_fulfilled'],
      dtype='object')


,CanonicalSMILES,CID,ROMol,tanimoto_maccs,dice_maccs,tanimoto_morgan,dice_morgan,MACCS_fp,pIC50_pred,IC50_pred,molecular_weight,n_hba,n_hbd,logp,TPSA,ro5_fulfilled
0,C1CC(CC(C1)O)N2C=C(C(=N2)C3=CC=NC=C3)C4=CC5=C(...,11531192,<rdkit.Chem.rdchem.Mol object at 0x00000156776...,0.803279,0.890909,0.578125,0.732673,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,10.081473,0.082895,388.189926,6,2,4.2126,83.53,True
1,C1CC(=NO)C2=C1C=C(C=C2)C3=CN(N=C3C4=CC=NC=C4)CCCO,44587190,<rdkit.Chem.rdchem.Mol object at 0x00000156776...,0.982143,0.990991,0.900000,0.947368,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,10.021318,0.095210,348.158626,6,2,3.1190,83.53,True
2,C1CNCCC1N2C=C(C(=N2)C3=CC=NC=C3)C4=CC5=C(C=C4)...,11653652,<rdkit.Chem.rdchem.Mol object at 0x00000156776...,0.816667,0.899083,0.606557,0.755102,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,10.020734,0.095338,373.190260,6,2,3.6611,75.33,True


In [8]:
# Add Hydrogens to molecules
mols = [Chem.MolFromSmiles(m) for m in df.CanonicalSMILES]
hmols = [Chem.AddHs(m) for m in mols]

# for mol  in hmols:
#     AllChem.EmbedMolecule(mol,AllChem.ETKDG()) #embed molecules using ETKDG conformer generation method (there are diff. embedding methods)
#     AllChem.UFFOptimizeMolecule(mol,1000) #clean using a force field - Universal Force Field (UFF)

#save info to variables
smiles = list(df.CanonicalSMILES)
cids = list(df.CID)

The following steps create the mol2 file that will be used by the docking program as a metabolite library:

5. Load the sdf file recently created to a Pandas Dataframe using Rdkit Pandas Tools
6. Save SMILES and Library Id into SMILES and LIBS variables (this step is redundant if you look at the previous code cell. But do it anyways incase you directly part from an sdf file ;))
7. Read molecules from SMILES and add a title to each one. I recommend you include an ID that can be traced back to the original metabolite database. It will make the results analysis step much easier. 
8. Create 3D coordinates: make3D adds H (we already have them, but there is no conflict) and performs a quick local optimization  with 50 steps and the MMFF94 forcefield. We want to further refine this optimization, hence why we use localopt with 500 steps. 

In [9]:
out=pybel.Outputfile(filename='docking_ligands.mol2',format='mol2',overwrite=True) #open file to read and write

for smi in smiles:
        mol=pybel.readstring(string=smi,format='smiles') #read mol from SMILE

for index, lib in enumerate(cids):
        mol.title='mol_'+str(lib)+'_'+str(index) #title = mol_libraryID_index

        mol.make3D('mmff94s') #write mols 3D coordinates
        mol.localopt(forcefield='mmff94s', steps=500) #Locally optimize the coordinates

        out.write(mol)
out.close()

Now to the even more fun part (lol):

## 3. Docking 

### 3.1. Docking coordinate calculation

There are many methods that can be used to calculate the exact coordinates to which the ligands will be docked to your protein. We will see 3 of them:

1. Manual calculation: this is not the best method as it depends on third-party predictions of putative binding residues like the ones offered by the I-TASSER server. 
2. fpocket: very easy to use, can be installed through conda
3. CB-Dock

#### 3.1.1. Manual calculation

You will need BioPython package for this step, more specifically the PDBParser module. Install the package through Conda:

```
    conda install -c conda-forge biopython
```

Then you will be ready to **parse** any PDB file you want!


In [26]:
### Get coordinates of the binding site center in Chain A (mean of the PDB_4xv2 ligand Dabrafenib)

from Bio.PDB.PDBParser import PDBParser
import statistics

parser = PDBParser(PERMISSIVE=1)
structure = parser.get_structure("Dabrafenib", "./files/Ligand_4xv2_ChA.pdb")
model = structure[0]
ligand = model.get_list()[0].get_list()[0]

def get_coords(PDB_ligand):
    parser = PDBParser(PERMISSIVE=1)
    structure = parser.get_structure("Dabrafenib", PDB_ligand)
    model = structure[0]
    ligand = model.get_list()[0].get_list()[0]
    
    coord_x=[]
    coord_y=[]
    coord_z=[]

    for atom in ligand:
        coord = (atom.get_coord())
        coord_x.append(coord[0])
        coord_y.append(coord[1])
        coord_z.append(coord[2])
                            
    mean_x = statistics.mean(coord_x)
    mean_y = statistics.mean(coord_y)
    mean_z = statistics.mean(coord_z)

    return(mean_x, mean_y, mean_z)

print("Binding center Chain A:", get_coords("./files/Ligand_4xv2_ChA.pdb"))
print("Binding center Chain B:", get_coords("./files/Ligand_4xv2_ChB.pdb"))

Binding center Chain A: (-0.3668, -3.1906571, -20.677143)
Binding center Chain B: (-3.8276572, -1.0898571, 6.133)
<Model id=0>


In [18]:
mean_x=0
mean_y=0
mean_z=0

for model in structure.get_list():
        for chain in model.get_list():
            coord_x=[]
            coord_y=[]
            coord_z=[]
            for residue in chain:
                for atom in residue:
                    coord = (atom.get_coord())
                    coord_x.append(coord[0])
                    coord_y.append(coord[1])
                    coord_z.append(coord[2])
                        
mean_x = statistics.mean(coord_x)
mean_y = statistics.mean(coord_y)
mean_z = statistics.mean(coord_z)

In [ ]:
coord_x
#open pdb file

In [ ]:
with open('Prot_manual_coords.txt', 'w') as f:
        f.write(f"{str(mean_x)} {str(mean_y)} {str(mean_z)} \n")

With the code above, you will add all coordinates of **all atoms** in the PDB file. You can also restrict the summing by giving it a string of residues (the ones you'd like to add the coordinates of). Note you can define the structure AND the chain you want to work with (if there are multiple chains, of course). 

Bellow, you will find a piece of code that explains how you can "navigate" through the different residues in a chain. You can think of a way to re-furbish the code above to parse a list of residues and calculate a mean of each alpha carbon on each residue (for example).


In [16]:
print(chain[10].has_id("CA"))
ca = chain[int(10)]["CA"]
print(ca)
coord = (ca.get_coord())
print(coord)

True
<Atom CA>
[46.205 58.655 62.679]


#### 3.1.2. fpocket

fpocket is a very fast cavitiy detection algorithm that can be installed through conda (either should work):

```
    conda install -c conda-forge fpocket
    conda install -c "conda-forge/label/cf202003" fpocket
```

Nonetheless fpocket is not designed to be a docking-cavity "searcher" and therefore will not give you the calculated coordinates and box-sizes. It will give you the coordinates of every atom (in each residue) that compose the pocket/cavity. You will need the code above to calculate the mean value of all atoms in the pocket/cavity to calculate the x, y and z coordinates and the box size can be set by default to 20x20x20.

#### 3.1.3. CB-Dock

This only works in Linux or macOS as it cannot be installed through conda. You will need to download the executable here: http://clab.labshare.cn/cb-dock/php/manual.php. Follow the Install CB-Dock instructions. Mgltools and vina can be installed through conda:

```
    conda install -c bioconda autodock-vina
    conda install -c bioconda mgltools

    ##other set up config

    ##run CB-dock
    perl ./prog/AutoBlindDock.pl [protein.pdb] [One_of_10_Mols_BEC.mol2] [5] [protein]

Result:
    Cavities        center_x  center_y  center_z  size_x  size_y  size_z  cavity_size
    1       51.729  58.674  53.967  13  13  7  512
    2       62.152  66.341  52.287  12  5  7  258
    3       57.100  54.396  72.251  5  5  7  156
    4       64.959  69.042  68.544  8  4  6  116
    5       64.199  49.741  62.892  4  4  7  103
```


Let me show you on the terminal!


### 3.2. Docking using Smina

Smina is run through a Linux or macOS executable file and just needs to be called adding the required and desired attributes. 

```
    cat $OUTPUT/protein/protein_CB_dock_coords.txt | while IFS=$' ' read x y z j k l m; do ./smina -r $INPUT/protein/protein_cleanH.pdb -l 10_Mols_BEC.mol2 
    -o $OUTPUT/protein/protein_smina.sdf --center_x "$x" --center_y "$y" --center_z "$z" --size_x "$j" --size_y "$k" --size_z "$l" --exhaustiveness 8 
    --num_modes 1 --seed 7683; done

```

As an output smina produces an sdf file containing the 3D coordinates of all docked molecules. You can then load this sdf file onto a pandas dataframe (as I have showed you before) and merge both lists by the library ID that is the comon aspect of both dataframes. Once you have done this, you should sort the docking results by value (most negative to most positive).

The way I like to complete this task is by creating a "cheatsheet" the information on each metabolite:


In [26]:
numbers = list(range(0,5)) #number of metabolites
numbers_ = [] #create name "mol_" + "number" from 0 to # of metabolites

for index, lib in enumerate(LIBS):
    name='mol_'+str(lib)+'_'+str(index) #molecule descriptor same as above
    numbers_.append(name)

all = pd.DataFrame(list(zip(SMILES, LIBS, numbers_))) #create a dataframe with all available info on each metabolite
all

,0,1,2
0,[H]Oc1c([H])c([H])c(C([H])([H])C([H])([H])C(=O...,12817,mol_12817_0
1,[H]Oc1c([H])c(O[H])c2c(c1[H])OC([H])(c1c([H])c...,16606,mol_16606_1
2,[H]Oc1c([H])c(O[H])c2c(c1[H])OC([H])(c1c([H])c...,14037,mol_14037_2
3,[H]Oc1c([H])c(-c2oc3c([H])c(O[H])c([H])c(O[H])...,15331,mol_15331_3
4,[H]Oc1c([H])c([H])c(-c2oc3c([H])c(O[H])c([H])c...,12635,mol_12635_4


Once I have this I can load the smina output file and merge both lists by the ID I gave each metabolite at the beginning. Make sure you always match the descriptors on the cheatsheet and on the initial metabolite database so you have something in common in both dataframes for your merge.

In [20]:
smina = PandasTools.LoadSDF('BEC_smina.sdf', embedProps=True, molColName=None, removeHs=False)
smina

,minimizedAffinity,ID
0,-9.04054,mol_12817_0
1,-10.51880,mol_16606_1
2,-10.44451,mol_14037_2
3,-10.50802,mol_15331_3
4,-9.87622,mol_12635_4


In [30]:
merge = pd.merge(smina, all, left_on='ID', right_on=2)
merge

,minimizedAffinity,ID,0,1,2
0,-9.04054,mol_12817_0,[H]Oc1c([H])c([H])c(C([H])([H])C([H])([H])C(=O...,12817,mol_12817_0
1,-10.51880,mol_16606_1,[H]Oc1c([H])c(O[H])c2c(c1[H])OC([H])(c1c([H])c...,16606,mol_16606_1
2,-10.44451,mol_14037_2,[H]Oc1c([H])c(O[H])c2c(c1[H])OC([H])(c1c([H])c...,14037,mol_14037_2
3,-10.50802,mol_15331_3,[H]Oc1c([H])c(-c2oc3c([H])c(O[H])c([H])c(O[H])...,15331,mol_15331_3
4,-9.87622,mol_12635_4,[H]Oc1c([H])c([H])c(-c2oc3c([H])c(O[H])c([H])c...,12635,mol_12635_4


In [32]:
merge = merge.astype({'minimizedAffinity': 'float'})
merge_sorted = merge.sort_values('minimizedAffinity')
merge_sorted

,minimizedAffinity,ID,0,1,2
1,-10.51880,mol_16606_1,[H]Oc1c([H])c(O[H])c2c(c1[H])OC([H])(c1c([H])c...,16606,mol_16606_1
3,-10.50802,mol_15331_3,[H]Oc1c([H])c(-c2oc3c([H])c(O[H])c([H])c(O[H])...,15331,mol_15331_3
2,-10.44451,mol_14037_2,[H]Oc1c([H])c(O[H])c2c(c1[H])OC([H])(c1c([H])c...,14037,mol_14037_2
4,-9.87622,mol_12635_4,[H]Oc1c([H])c([H])c(-c2oc3c([H])c(O[H])c([H])c...,12635,mol_12635_4
0,-9.04054,mol_12817_0,[H]Oc1c([H])c([H])c(C([H])([H])C([H])([H])C(=O...,12817,mol_12817_0


You can also merge the output result to your initial metabolite database which will probably have some more information on each metabolite. This time you can merge by Library ID (both dataframes have this). 

I leave that to you :)